In [91]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time, requests
import json
import numpy as np
import mysql.connector as mysql
from currency_converter import CurrencyConverter

In [92]:
import undetected_chromedriver as uc
driver = uc.Chrome()
driver.maximize_window()

In [93]:
c = CurrencyConverter()

In [94]:
mydb = mysql.connect(
  host="l6glqt8gsx37y4hs.cbetxkdyhwsb.us-east-1.rds.amazonaws.com",
  user="vhaithj2ezjri3a5",
  password="htf06l6xp0gsgoei",
  database = "bovesv3pjmkwzex5"
)

In [95]:
# driver=webdriver.Chrome(executable_path='D:\chromedriver.exe')
# driver.maximize_window()


In [96]:
# mail_address = 'windwalkerz42@gmail.com'
# password = '@Best'
# url = 'https://www.google.com/accounts/Login?hl=ja&continue=http://www.google.co.jp/'
# driver.get(url)

USD

In [97]:
def convertUsd(price):
    price = c.convert(price, 'USD', 'THB')
    return price

EUR

In [98]:
def convertEur(price):
    price = c.convert(price, 'EUR', 'THB')
    return price
    

  *** Function market price ***

GameSrig

In [99]:
def market_8(url, gameId, marketId):
    driver.get(url)
    try:
        findPrice = driver.find_element_by_xpath('//*[@id="blockB"]/div/div/div[2]/div').text
        price = float(findPrice.replace("฿",""))
    except:
        price = float(0)
    update = "UPDATE price_check SET now_price = (%s) WHERE game_id  = (%s) AND market_id = (%s)"
    values = [price, gameId, marketId]
    cursor.execute(update,values)
    mydb.commit()
    cursor.close

HRK

In [100]:
def market_9(url, gameId, marketId):
    driver.get(url)
    try:
        findPrice = driver.find_element_by_xpath('//*[@id="store_form"]/div/div[1]/div/div/div[2]')
        beforePrice = float(findPrice.text.strip("$"))
    except:
        lastPrice = float(0)
        beforePrice = float(0)
    lastPrice = round(convertUsd(beforePrice),2)
    lastPrice = str(lastPrice)
    beforePrice = str(beforePrice)
    update = "UPDATE price_check SET before_price = (%s), now_price = (%s) WHERE game_id  = (%s) AND market_id = (%s)"
    values = [beforePrice,lastPrice, gameId, marketId]
    cursor.execute(update,values)
    mydb.commit()
    cursor.close


Kinguin

In [101]:
def market_10(url, gameId, marketId):
    driver.get(url)
    try:   
        findPrice = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div[1]/div[1]/div[2]/div[2]/div[3]/div/div/div[4]/div/div[2]/div/div/span[1]/span[2]').text
        beforePrice = float(findPrice.strip("€ "))
    except:
        lastPrice = float(0)
        beforePrice = float(0)
    lastPrice = round(convertEur(beforePrice),2)
    lastPrice = str(lastPrice)
    beforePrice = str(beforePrice)
    update = "UPDATE price_check SET before_price = (%s), now_price = (%s) WHERE game_id  = (%s) AND market_id = (%s)"
    values = [beforePrice,lastPrice, gameId, marketId]
    cursor.execute(update,values)
    mydb.commit()
    cursor.close

Steam price

In [102]:
def market_11(url, gameId, marketId):
    cursor = mydb.cursor()
    cursor.execute("SELECT steam_id FROM games_data WHERE game_id = "+str(gameId)+"")
    myresult = cursor.fetchall()
    response = requests.get(url)
    gameJson = response.json()
    price = 0
    try :
        if (gameJson[myresult[0][0]]['success'] == True) :
            gameDetail = gameJson[myresult[0][0]]['data']
            price = float(gameDetail['price_overview']['final_formatted'].replace("฿","").replace(",",""))
    except:
        print(myresult[0][0])
    finally:
        cursor = mydb.cursor()
        update = "UPDATE price_check SET now_price = %s WHERE game_id  = %s AND market_id = %s"
        values = [price,gameId,marketId]
        cursor.execute(update,values)
        mydb.commit() 
        cursor.close
    

Epic Games

In [103]:
def market_12(url, gameId, marketId, checkEpicgames):
    driver.get(url)
    try:
        if checkEpicgames == 0:
            driver.find_element_by_class_name('css-38hacf').click()
            checkEpicgames = True
    except:
        price = float(0)
        beforePrice = float(0)
    time.sleep(2)
    findPrice = driver.find_element_by_xpath('//*[@id="dieselReactWrapper"]/div/div[4]/main/div[2]/div/div/div/div[2]/div/div/aside/div/div/div[3]/div/div/span/div/span').text
    price =  float(findPrice.replace("THB","").replace(",",""))
    cursor = mydb.cursor()
    update = "UPDATE price_check SET now_price = %s WHERE game_id  = %s AND market_id = %s"
    values = [price, gameId, marketId]
    cursor.execute(update,values)
    mydb.commit()
    cursor.close
        


Gamivo

In [104]:
def market_13(url, gameId, marketId):
    driver.get(url)
    try:
        time.sleep(2)
        findPrice = driver.find_element_by_xpath('/html/body/app-root/div/div/app-product/div/div[2]/div[2]/app-product-offer-price/div/div[4]/div[1]/div/div[2]/span').text
        beforePrice = float(findPrice.strip("€ "))
        print(beforePrice)
        
    except:
        lastPrice = float(0)
        beforePrice = float(0)
    lastPrice = round(convertEur(beforePrice),2)
    lastPrice = str(lastPrice)
    beforePrice = str(beforePrice)
    cursor = mydb.cursor()
    update = "UPDATE price_check SET before_price = (%s), now_price = (%s) WHERE game_id  = (%s) AND market_id = (%s)"
    values = [beforePrice,lastPrice, gameId, marketId]
    cursor.execute(update,values)
    mydb.commit()
    cursor.close

In [105]:
cursor = mydb.cursor()
cursor.execute("SELECT game_id, market_id, market_url  FROM price_check")
myresult = cursor.fetchall()
checkEpicgames = 0
for i in myresult:
        print(i)
        if i[1] == 8:
            url = i[2]
            gameId = i[0]
            marketId = i[1]
            market_8(url, gameId, marketId)
        elif i[1] == 9:
            url = i[2]
            gameId = i[0]
            marketId = i[1]
            market_9(url, gameId, marketId)
        elif i[1] == 10:
            url = i[2]
            gameId = i[0]
            marketId = i[1]
            market_10(url, gameId, marketId)
        elif i[1] == 11:
            url = i[2]
            gameId = i[0]
            marketId = i[1]
            market_11(url, gameId, marketId)
        elif i[1] ==  12:
            url = i[2]
            gameId = i[0]
            marketId = i[1]
            market_12(url, gameId, marketId, checkEpicgames)
            checkEpicgames += 1
        elif i[1] ==  13:
            url = i[2]
            gameId = i[0]
            marketId = i[1]
            market_13(url, gameId, marketId)
mydb.commit()


(22, 8, 'https://www.gamesrig.com/god-of-war-steam-key')


C:\Users\BoraBestz\AppData\Local\Temp\ipykernel_4192\176298565.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  findPrice = driver.find_element_by_xpath('//*[@id="blockB"]/div/div/div[2]/div').text


(22, 9, 'https://www.hrkgame.com/en/games/product/god-of-war')
(22, 10, 'https://www.kinguin.net/category/108174/god-of-war-steam-cd-key')
(22, 11, 'https://store.steampowered.com/api/appdetails?appids=1593500&l=thai')
(22, 12, 'https://www.epicgames.com/store/en-US/p/god-of-war')


C:\Users\BoraBestz\AppData\Local\Temp\ipykernel_4192\3814315525.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_class_name('css-38hacf').click()
C:\Users\BoraBestz\AppData\Local\Temp\ipykernel_4192\3814315525.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  findPrice = driver.find_element_by_xpath('//*[@id="dieselReactWrapper"]/div/div[4]/main/div[2]/div/div/div/div[2]/div/div/aside/div/div/div[3]/div/div/span/div/span').text


(22, 13, 'https://www.gamivo.com/product/god-of-war-steam')
37.99
(49, 8, 'https://www.gamesrig.com/en/all-products/dying-light-2-steam-key')
(49, 9, 'https://www.hrkgame.com/en/games/product/dying-light-2-stay-human-global')
(49, 10, 'https://www.kinguin.net/category/109526/dying-light-2-steam-cd-key')


UnboundLocalError: local variable 'beforePrice' referenced before assignment

In [ ]:
mydb.close()